1/ Importation des données TC et TITMC 

2/ Import et traitement du fichier des prénoms 

3/ Ajouter le genre des personnes 

4/ Codification des adresses 

5/ Ajout et traitement du code commune

6/ Retraiter et simplifier les qualités 

In [ ]:
# Lecture de la base inpi
import os
import s3fs
import pandas as pd
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "radjerad/inpi"
FILE_KEY_S3 = "data_rep_pm_2017.csv"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    df_rep_pm = pd.read_csv(file_in, sep=";", encoding="utf-8")

In [ ]:
df_rep_pm.shape # 9 751 833

In [ ]:
df_rep_pm = df_rep_pm.rename({"code greffe": "code_greffe"}, axis=1)

In [ ]:
df_rep_pm = df_rep_pm.loc[:, ["code_greffe", "siren", "denomination", "forme_juridique_x", "qualité",
"nom_patronymique", "nom_usage", "prénoms", "type", "date_naissance", "ville_naissance", "adresse_ligne1", "adresse_ligne2",
"adresse_ligne3", "code_postal", "ville", "code_commune", "pays", "id_représentant"]]

In [ ]:
df_rep_pm = df_rep_pm.rename({"forme_juridique_x": "forme_juridique", "prénoms": "prenoms", "qualité": "qualite", "id_représentant": "id_representant"}, axis=1)

In [ ]:
df_rep_pm.head(10)

In [ ]:
df_rep_pm["qualite"].isna().sum() # 246 926

In [31]:
# Traitement des cas où le nom et les prénoms sont manquants

re2 = "(\sNEE\s[A-Za-z\-]+)\s([A-Za-z\-\.\s]+)"


In [ ]:
df_rep_pm.loc[df_rep_pm["prenoms"] == "-", "prenoms"] = ""

In [11]:
df_rep_pm.loc[df_rep_pm["prenoms"] == "", :].shape # 4 448

(4448, 19)

In [13]:
df_rep_pm.loc[(df_rep_pm["prenoms"] == "") & (df_rep_pm["type"] == "P.Physique"), :].shape # 4 448

(4448, 19)

In [41]:
re1 = "(\sM\.\s|^M\.\s|MME\.\s|MR\.\s|MELLE\.\s|MONSIEUR\.?\s|MADAME\.?\s|MLLE\.\s)"
df_rep_pm["top_civilite"] = df_rep_pm["nom_patronymique"].str.contains(re1, regex=True)

/tmp/ipykernel_2360/3704456014.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_rep_pm["top_civilite"] = df_rep_pm["nom_patronymique"].str.contains(re1, regex=True)


In [42]:
df_rep_pm.loc[df_rep_pm["top_civilite"] == True, "nom_patronymique"].head()

151606    INDIVISION SUCCESSORALE DE M. DE VILLARDI DE M...
158708    INDIVISION SUCCESSORALE DE M. DE LAITRE JEAN P...
320046    C.I.I.M. REPRESENTEE PAR MONSIEUR LAVIGUEUR BE...
320809    SARL S.A.F. REPRESENTEE PAR SA GERANTE MADAME ...
322994    SARL EURO DELTA IMMOBILIER REPRESENTEE PAR MAD...
Name: nom_patronymique, dtype: object

In [26]:
temp.shape

(8671190,)